# Data and Data storage



## Document and management

In [1]:
query_1 = "What are the benefits of renewable energy?" # gt is [0, 3]
query_2 = "How do solar panels impact the environment?" # gt is [1, 2]

org_documents =[
    {
        "title": "The Impact of Renewable Energy on the Economy",
        "content": "Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure."
    },
    {
        "title": "Understanding Solar Panels",
        "content": "Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock electrons free from atoms, generating a flow of electricity. Solar panels are a type of renewable energy technology that has been found to have a significant positive effect on the environment by reducing the reliance on fossil fuels."
    },
    {
        "title": "Pros and Cons of Solar Energy",
        "content": "While solar energy offers substantial environmental benefits, such as reducing carbon footprints and pollution, it also has downsides. The production of solar panels can lead to hazardous waste, and large solar farms require significant land, which can disrupt local ecosystems."
    },
    {
        "title":  "Renewable Energy and Its Effects",
        "content": "Renewable energy sources like wind, solar, and hydro power play a crucial role in combating climate change. They do not produce greenhouse gases during operation, making them essential for sustainable development. However, the initial setup and material sourcing for these technologies can still have environmental impacts."
    }
]

In [2]:
from lightrag.core.types import Document

# we will save the content to text and title in the meta_data
documents  = [Document(text=doc['content'], meta_data={'title': doc['title']}) for doc in org_documents]
print(documents)

[Document(id=a38904a0-7700-4789-9b66-f5c67fc016bd, text='Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute...', meta_data={'title': 'The Impact of Renewable Energy on the Economy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=29fd0f84-113d-4727-84bd-c1f8bd65f20a, text='Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock ele...', meta_data={'title': 'Understanding Solar Panels'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=798f77e9-4491-4e91-beb2-17c959797049, text='While solar energy offers substantial environmental benefits, such as reducing carbon footprints and...', meta_data={'title': 'Pros and Cons of Solar Energy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=b6ce43af-2f85-43a7-95e3-6242e3790df3, text='Renewable energy sources like wind, solar, and hydro power play a crucial role in combating climate ...', meta_dat

In [3]:
# do dialogturns 

from lightrag.core.types import DialogTurn

turns = [
    {
        "user": "What are the benefits of renewable energy?",
        "system": "I can see you are interested in renewable energy. Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.",
        "user_time": "2021-09-01T12:00:00Z",
        "system_time": "2021-09-01T12:00:01Z"
    },
    {
        "user": "How do solar panels impact the environment?",
        "system": "Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock electrons free from atoms, generating a flow of electricity. Solar panels are a type of renewable energy technology that has been found to have a significant positive effect on the environment by reducing the reliance on fossil fuels.",
        "user_time": "2021-09-01T12:00:02Z",
        "system_time": "2021-09-01T12:00:03Z"
    }
]

dialog_turns = [DialogTurn(user_query = turn['user'], assistant_response = turn['system'], user_query_timestamp = turn['user_time'], assistant_response_timestamp = turn['system_time']) for turn in turns]
print(dialog_turns)

[DialogTurn(id='979c0f92-6255-4e7c-8299-adf04e9aac34', user_id=None, session_id=None, order=None, user_query='What are the benefits of renewable energy?', assistant_response='I can see you are interested in renewable energy. Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.', user_query_timestamp='2021-09-01T12:00:00Z', assistant_response_timestamp='2021-09-01T12:00:01Z', metadata=None, vector=None), DialogTurn(id='e0de6e3a-641b-47f6-86cd-f52f1c2e0372', user_id=None, session_id=None, order=None, user_query='How do solar panels impact the environment?', assistant_response='Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock electrons free from atoms, generating a flow of electrici

In [4]:
# use LocalDB to assist the CRUD of the documents and the persisting of the documents

from lightrag.core.db import LocalDB

db = LocalDB(  )
print(db)

LocalDB(name=None, items=[], transformed_items={}, mapped_items={}, transformer_setups={}, mapper_setups={})


### Loading and CRUD operations on DialogTurns



In [5]:
# prepare the data pipeline

from lightrag.core.embedder import Embedder 
from lightrag.core.types import ModelClientType
from lightrag.components.data_process import DocumentSplitter, ToEmbeddings
from lightrag.core.component import Sequential


model_kwargs = {
    "model": "text-embedding-3-small",
    "dimensions": 256,
    "encoding_format": "float",
}

splitter_config = {
    "split_by": "word",
    "split_length": 50,
    "split_overlap": 10
}

splitter = DocumentSplitter(**splitter_config)
embedder = Embedder(model_client =ModelClientType.OPENAI(), model_kwargs=model_kwargs)
embedder_transformer = ToEmbeddings(embedder, batch_size=2)
data_transformer = Sequential(splitter, embedder_transformer)
print(data_transformer)

Sequential(
  (0): DocumentSplitter(split_by=word, split_length=50, split_overlap=10)
  (1): ToEmbeddings(
    batch_size=2
    (embedder): Embedder(
      model_kwargs={'model': 'text-embedding-3-small', 'dimensions': 256, 'encoding_format': 'float'}, 
      (model_client): OpenAIClient()
    )
    (batch_embedder): BatchEmbedder(
      (embedder): Embedder(
        model_kwargs={'model': 'text-embedding-3-small', 'dimensions': 256, 'encoding_format': 'float'}, 
        (model_client): OpenAIClient()
      )
    )
  )
)


In [6]:
# prepare mapping functions to map the data to documents for the pipeline

from typing import Dict
# mapping function for org_documents
def map_to_document(doc: Dict) -> Document:
    return Document(text=doc['content'], meta_data={'title': doc['title']})

def map_dialogturn_to_document(turn: DialogTurn) -> Document:
    # it can be important to keep the original data's id
    return Document(id=turn.id, text=turn.user_query + ' ' + turn.assistant_response)

In [7]:
# apply data transformation to the dialog_turns

dialog_turns_as_documents = [map_dialogturn_to_document(turn) for turn in dialog_turns]
print(dialog_turns_as_documents)

# apply data transformation to the documents
output = data_transformer(dialog_turns_as_documents)
print(output)

[Document(id=979c0f92-6255-4e7c-8299-adf04e9aac34, text='What are the benefits of renewable energy? I can see you are interested in renewable energy. Renewab...', meta_data=None, vector=[], parent_doc_id=None, order=None, score=None), Document(id=e0de6e3a-641b-47f6-86cd-f52f1c2e0372, text='How do solar panels impact the environment? Solar panels convert sunlight into electricity by allowi...', meta_data=None, vector=[], parent_doc_id=None, order=None, score=None)]


Batch embedding documents:   0%|          | 0/2 [00:00<?, ?it/s]

Batch embedding documents: 100%|██████████| 2/2 [00:00<00:00,  3.22it/s]
Adding embeddings to documents from batch: 2it [00:00, 10381.94it/s]

[Document(id=10f79974-24bc-40aa-8c7c-908fe39657f4, text='What are the benefits of renewable energy? I can see you are interested in renewable energy. Renewab...', meta_data=None, vector='len: 256', parent_doc_id=979c0f92-6255-4e7c-8299-adf04e9aac34, order=0, score=None), Document(id=e98d1d83-0915-44f2-b677-b6d99572b1a6, text='and installation sectors. The growth in renewable energy usage boosts local economies through increa...', meta_data=None, vector='len: 256', parent_doc_id=979c0f92-6255-4e7c-8299-adf04e9aac34, order=1, score=None), Document(id=0b978d41-680b-4388-a220-3f905717e489, text='How do solar panels impact the environment? Solar panels convert sunlight into electricity by allowi...', meta_data=None, vector='len: 256', parent_doc_id=e0de6e3a-641b-47f6-86cd-f52f1c2e0372, order=0, score=None), Document(id=6b4b2977-303c-4b7a-a233-f0f699ac7355, text='has been found to have a significant positive effect on the environment by reducing the reliance on ...', meta_data=None, vector='

In [8]:
from lightrag.core.db import LocalDB

dialog_turn_db = LocalDB('dialog_turns')
print(dialog_turn_db)

dialog_turn_db.load(dialog_turns)
print(dialog_turn_db)

LocalDB(name='dialog_turns', items=[], transformed_items={}, mapped_items={}, transformer_setups={}, mapper_setups={})
LocalDB(name='dialog_turns', items=[DialogTurn(id='979c0f92-6255-4e7c-8299-adf04e9aac34', user_id=None, session_id=None, order=None, user_query='What are the benefits of renewable energy?', assistant_response='I can see you are interested in renewable energy. Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.', user_query_timestamp='2021-09-01T12:00:00Z', assistant_response_timestamp='2021-09-01T12:00:01Z', metadata=None, vector=None), DialogTurn(id='e0de6e3a-641b-47f6-86cd-f52f1c2e0372', user_id=None, session_id=None, order=None, user_query='How do solar panels impact the environment?', assistant_response='

In [9]:
# apply data transformation to the dialog_turn_db

key = "split_and_embed"
dialog_turn_db.transform_data(data_transformer, map_fn=map_dialogturn_to_document, key=key)
print(dialog_turn_db.transformed_items[key])
print(dialog_turn_db.transformer_setups[key])

Batch embedding documents: 100%|██████████| 2/2 [00:00<00:00,  5.27it/s]
Adding embeddings to documents from batch: 2it [00:00, 36314.32it/s]

[Document(id=46258e1d-1cb0-46ac-b031-427eb4aed4a9, text='What are the benefits of renewable energy? I can see you are interested in renewable energy. Renewab...', meta_data=None, vector='len: 256', parent_doc_id=979c0f92-6255-4e7c-8299-adf04e9aac34, order=0, score=None), Document(id=893892d6-9d76-4a9e-b7a7-cf1d82b4f1e5, text='and installation sectors. The growth in renewable energy usage boosts local economies through increa...', meta_data=None, vector='len: 256', parent_doc_id=979c0f92-6255-4e7c-8299-adf04e9aac34, order=1, score=None), Document(id=77fd5b06-7345-4090-8a43-e4454bca7c8c, text='How do solar panels impact the environment? Solar panels convert sunlight into electricity by allowi...', meta_data=None, vector='len: 256', parent_doc_id=e0de6e3a-641b-47f6-86cd-f52f1c2e0372, order=0, score=None), Document(id=7d777c13-9cdd-4ed8-bf86-fa502d61144f, text='has been found to have a significant positive effect on the environment by reducing the reliance on ...', meta_data=None, vector='

In [10]:
# test seraialization and deserialization of the data

transformer = Sequential(splitter)
test_db = LocalDB('test')
test_db.load(dialog_turns)

key = "split"
test_db.transform_data(transformer, map_fn=map_dialogturn_to_document, key=key)
print(test_db.transformed_items[key])
test_db.pickle_to_file('test_db_state.pkl')
restored_test_db = LocalDB.load_from_pickle('test_db_state.pkl')

print(restored_test_db.transformed_items[key])
print


Splitting documents: 100%|██████████| 2/2 [00:00<00:00, 1370.91it/s]

[Document(id=38ae771b-124e-4b49-b564-647ab2bd1050, text='What are the benefits of renewable energy? I can see you are interested in renewable energy. Renewab...', meta_data=None, vector=[], parent_doc_id=979c0f92-6255-4e7c-8299-adf04e9aac34, order=0, score=None), Document(id=6d01b89b-6fbc-4579-b4e4-8bbe4dcfdef4, text='and installation sectors. The growth in renewable energy usage boosts local economies through increa...', meta_data=None, vector=[], parent_doc_id=979c0f92-6255-4e7c-8299-adf04e9aac34, order=1, score=None), Document(id=17301d35-220e-46de-9d89-c0e75e54bf10, text='How do solar panels impact the environment? Solar panels convert sunlight into electricity by allowi...', meta_data=None, vector=[], parent_doc_id=e0de6e3a-641b-47f6-86cd-f52f1c2e0372, order=0, score=None), Document(id=bd0af9d9-5821-4c86-b1ad-46c4dd9e3646, text='has been found to have a significant positive effect on the environment by reducing the reliance on ...', meta_data=None, vector=[], parent_doc_id=e0de6e3

In [13]:
dialog_turn_db.pickle_to_file(filepath='.storage/dialog_turns.pkl')
print(dialog_turn_db)

LocalDB(name='dialog_turns', items=[DialogTurn(id='979c0f92-6255-4e7c-8299-adf04e9aac34', user_id=None, session_id=None, order=None, user_query='What are the benefits of renewable energy?', assistant_response='I can see you are interested in renewable energy. Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.', user_query_timestamp='2021-09-01T12:00:00Z', assistant_response_timestamp='2021-09-01T12:00:01Z', metadata=None, vector=None), DialogTurn(id='e0de6e3a-641b-47f6-86cd-f52f1c2e0372', user_id=None, session_id=None, order=None, user_query='How do solar panels impact the environment?', assistant_response='Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock electrons free from at

In [12]:
reloaded_dialog_turn_db = LocalDB.load_from_pickle(filepath='.storage/dialog_turns.pkl')
print(reloaded_dialog_turn_db)

LocalDB(name='dialog_turns', items=[{'id': '979c0f92-6255-4e7c-8299-adf04e9aac34', 'user_id': None, 'session_id': None, 'order': None, 'user_query': 'What are the benefits of renewable energy?', 'assistant_response': 'I can see you are interested in renewable energy. Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.', 'user_query_timestamp': '2021-09-01T12:00:00Z', 'assistant_response_timestamp': '2021-09-01T12:00:01Z', 'metadata': None, 'vector': None}, {'id': 'e0de6e3a-641b-47f6-86cd-f52f1c2e0372', 'user_id': None, 'session_id': None, 'order': None, 'user_query': 'How do solar panels impact the environment?', 'assistant_response': 'Solar panels convert sunlight into electricity by allowing photons, or light particles, to 

In [ ]:
# TODO: save the state of transform setup as a to_dict method

### Loading and CRUD operations on documents

In [5]:
db.load(documents)
print(db)

LocalDB(name=None, items=[Document(id=6583698e-c676-4966-a0a3-91a0f3de3796, text='Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute...', meta_data={'title': 'The Impact of Renewable Energy on the Economy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=fab80c66-5da8-4639-9431-3f64f5829ab1, text='Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock ele...', meta_data={'title': 'Understanding Solar Panels'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=204e97ed-bd37-4d6f-b14e-e7518b4d1dc8, text='While solar energy offers substantial environmental benefits, such as reducing carbon footprints and...', meta_data={'title': 'Pros and Cons of Solar Energy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=880bf5f9-7dde-4d4c-af74-83bdc82cbfba, text='Renewable energy sources like wind, solar, and hydro power play a crucial role in combati

In [6]:
# add and delete documents

new_document = Document(
    text="This is a new document", meta_data={'title': 'New Document'})

db.add(new_document)
print(db)

LocalDB(name=None, items=[Document(id=6583698e-c676-4966-a0a3-91a0f3de3796, text='Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute...', meta_data={'title': 'The Impact of Renewable Energy on the Economy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=fab80c66-5da8-4639-9431-3f64f5829ab1, text='Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock ele...', meta_data={'title': 'Understanding Solar Panels'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=204e97ed-bd37-4d6f-b14e-e7518b4d1dc8, text='While solar energy offers substantial environmental benefits, such as reducing carbon footprints and...', meta_data={'title': 'Pros and Cons of Solar Energy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=880bf5f9-7dde-4d4c-af74-83bdc82cbfba, text='Renewable energy sources like wind, solar, and hydro power play a crucial role in combati

In [7]:
db.delete(indices=[-1])

In [8]:
len(db.documents)

AttributeError: 'LocalDB' object has no attribute 'documents'

In [ ]:
db.add(new_document, index=1)
print(db)
len(db.documents)

LocalDB(documents=[Document(id=b349a519-1139-42cb-9362-aa4a6a6919c4, text='Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute...', meta_data={'title': 'The Impact of Renewable Energy on the Economy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=49e80879-5b41-4f81-a236-ec512f2c60ad, text='This is a new document', meta_data={'title': 'New Document'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=12d12f6e-e269-4482-8b5b-7438a12d89b0, text='Solar panels convert sunlight into electricity by allowing photons, or light particles, to knock ele...', meta_data={'title': 'Understanding Solar Panels'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=3ed98cb6-1ee2-4a96-b7fb-94bc5dcd3913, text='While solar energy offers substantial environmental benefits, such as reducing carbon footprints and...', meta_data={'title': 'Pros and Cons of Solar Energy'}, vector=[], parent_doc_id=None, ord

5

In [ ]:
db.delete(indices=[1])
len(db.documents)

4

In [ ]:
db.delete(indices=[1])
len(db.documents)

3

In [ ]:
db

LocalDB(documents=[Document(id=b349a519-1139-42cb-9362-aa4a6a6919c4, text='Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute...', meta_data={'title': 'The Impact of Renewable Energy on the Economy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=3ed98cb6-1ee2-4a96-b7fb-94bc5dcd3913, text='While solar energy offers substantial environmental benefits, such as reducing carbon footprints and...', meta_data={'title': 'Pros and Cons of Solar Energy'}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=7e13dac8-3a2f-4dcb-aeba-2941d2249e1d, text='Renewable energy sources like wind, solar, and hydro power play a crucial role in combating climate ...', meta_data={'title': 'Renewable Energy and Its Effects'}, vector=[], parent_doc_id=None, order=None, score=None)], transformed_documents={}, mapped_documents={}, transformer_setups={})

## data transformation (processing)

In [ ]:
from lightrag.core.base_data_class import DataClassFormatType
first_document = db.documents[0]
json_str = first_document.format_example_str(DataClassFormatType.EXAMPLE_JSON)
print(json_str)

{
    "text": "Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure.",
    "meta_data": {
        "title": "The Impact of Renewable Energy on the Economy"
    },
    "vector": [],
    "id": "b349a519-1139-42cb-9362-aa4a6a6919c4",
    "order": "None",
    "score": "None",
    "parent_doc_id": "None",
    "estimated_num_tokens": 47
}


In [ ]:
yaml_str = first_document.format_example_str(DataClassFormatType.EXAMPLE_YAML)
print(yaml_str)

text: "Renewable energy technologies not only help in reducing greenhouse gas emissions but also contribute significantly to the economy by creating jobs in the manufacturing and installation sectors. The growth in renewable energy usage boosts local economies through increased investment in technology and infrastructure."
meta_data: title: The Impact of Renewable Energy on the Economy
vector: []
id: "b349a519-1139-42cb-9362-aa4a6a6919c4"
order: null
score: null
parent_doc_id: null
estimated_num_tokens: 47
